<a href="https://colab.research.google.com/github/T-Sawao/diveintocode-ml/blob/master/term2_sprint10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 2.ディープニューラルネットワークスクラッチ

前回は3層のニューラルネットワークを作成しましたが、今回はこれを任意の層数に拡張しやすいものに書き換えていきます。その上で、活性化関数や初期値、最適化手法について発展的なものを扱えるようにしていきます。


このようなスクラッチを行うことで、今後各種フレームワークを利用していくにあたり、内部の動きが想像できることを目指します。


名前は新しくScratchDeepNeuralNetworkClassifierクラスとしてください。


**層などのクラス化**  
クラスにまとめて行くことで、構成を変更しやすい実装にしていきます。


**手を加える箇所**


- 層の数
- 層の種類（今後畳み込み層など他のタイプの層が登場する）
- 活性化関数の種類
- 重みやバイアスの初期化方法
- 最適化手法

そのために、全結合層、各種活性化関数、重みやバイアスの初期化、最適化手法それぞれのクラスを作成します。


実装方法は自由ですが、簡単な例を紹介します。サンプルコード1のように全結合層と活性化関数のインスタンスを作成し、サンプルコード2,3のようにして使用します。それぞれのクラスについてはこのあと解説します。


《サンプルコード1》


ScratchDeepNeuralNetworkClassifierのfitメソッド内

In [ ]:
# self.sigma : ガウス分布の標準偏差
# self.lr : 学習率
# self.n_nodes1 : 1層目のノード数
# self.n_nodes2 : 2層目のノード数
# self.n_output : 出力層のノード数
# optimizer = SGD(self.lr)
# self.FC1 = FC(self.n_features, self.n_nodes1, SimpleInitializer(self.sigma), optimizer)
# self.activation1 = Tanh()
# self.FC2 = FC(self.n_nodes1, self.n_nodes2, SimpleInitializer(self.sigma), optimizer)
# self.activation2 = Tanh()
# self.FC3 = FC(self.n_nodes2, self.n_output, SimpleInitializer(self.sigma), optimizer)
# self.activation3 = Softmax()

《サンプルコード2》


イテレーションごとのフォワード

In [ ]:
# A1 = self.FC1.forward(X)
# Z1 = self.activation1.forward(A1)
# A2 = self.FC2.forward(Z1)
# Z2 = self.activation2.forward(A2)
# A3 = self.FC3.forward(Z2)
# Z3 = self.activation3.forward(A3)

《サンプルコード3》


イテレーションごとのバックワード



In [ ]:
# dA3 = self.activation3.backward(Z3, Y) # 交差エントロピー誤差とソフトマックスを合わせている
# dZ2 = self.FC3.backward(dA3)
# dA2 = self.activation2.backward(dZ2)
# dZ1 = self.FC2.backward(dA2)
# dA1 = self.activation1.backward(dZ1)
# dZ0 = self.FC1.backward(dA1) # dZ0は使用しない

## 【問題1】全結合層のクラス化
全結合層のクラス化を行なってください。


以下に雛形を載せました。コンストラクタで重みやバイアスの初期化をして、あとはフォワードとバックワードのメソッドを用意します。重みW、バイアスB、およびフォワード時の入力Xをインスタンス変数として保持しておくことで、煩雑な入出力は不要になります。


なお、インスタンスも引数として渡すことができます。そのため、初期化方法のインスタンスinitializerをコンストラクタで受け取れば、それにより初期化が行われます。渡すインスタンスを変えれば、初期化方法が変えられます。


また、引数として自身のインスタンスselfを渡すこともできます。これを利用してself.optimizer.update(self)という風に層の重みの更新が可能です。更新に必要な値は複数ありますが、全て全結合層が持つインスタンス変数にすることができます。


初期化方法と最適化手法のクラスについては後述します。


《雛形》

In [ ]:
class FC:
    """
    ノード数n_nodes1からn_nodes2への全結合層
    Parameters
    ----------
    n_nodes1 : int
      前の層のノード数
    n_nodes2 : int
      後の層のノード数
    initializer : 初期化方法のインスタンス
    optimizer : 最適化手法のインスタンス
    """
    def __init__(self, n_nodes1, n_nodes2, initializer, optimizer):
        self.optimizer = optimizer
        # 初期化
        # initializerのメソッドを使い、self.Wとself.Bを初期化する
        pass
    def forward(self, X):
        """
        フォワード
        Parameters
        ----------
        X : 次の形のndarray, shape (batch_size, n_nodes1)
            入力
        Returns
        ----------
        A : 次の形のndarray, shape (batch_size, n_nodes2)
            出力
        """        
        pass
        return A
    def backward(self, dA):
        """
        バックワード
        Parameters
        ----------
        dA : 次の形のndarray, shape (batch_size, n_nodes2)
            後ろから流れてきた勾配
        Returns
        ----------
        dZ : 次の形のndarray, shape (batch_size, n_nodes1)
            前に流す勾配
        """
        pass
        # 更新
        self = self.optimizer.update(self)
        return dZ

## 【問題2】初期化方法のクラス化
初期化を行うコードをクラス化してください。


前述のように、全結合層のコンストラクタに初期化方法のインスタンスを渡せるようにします。以下の雛形に必要なコードを書き加えていってください。標準偏差の値（sigma）はコンストラクタで受け取るようにすることで、全結合層のクラス内にこの値（sigma）を渡さなくてすむようになります。


これまで扱ってきた初期化方法はSimpleInitializerクラスと名付けることにします。


《雛形》

In [ ]:
class SimpleInitializer:
    """
    ガウス分布によるシンプルな初期化
    Parameters
    ----------
    sigma : float
      ガウス分布の標準偏差
    """
    def __init__(self, sigma):
        self.sigma = sigma
    def W(self, n_nodes1, n_nodes2):
        """
        重みの初期化
        Parameters
        ----------
        n_nodes1 : int
          前の層のノード数
        n_nodes2 : int
          後の層のノード数

        Returns
        ----------
        W :
        """
        pass
        return W
    def B(self, n_nodes2):
        """
        バイアスの初期化
        Parameters
        ----------
        n_nodes2 : int
          後の層のノード数

        Returns
        ----------
        B :
        """
        pass
        return B

## 【問題3】最適化手法のクラス化
最適化手法のクラス化を行なってください。


最適化手法に関しても初期化方法同様に全結合層にインスタンスとして渡します。バックワードのときにself.optimizer.update(self)のように更新できるようにします。以下の雛形に必要なコードを書き加えていってください。


これまで扱ってきた最適化手法はSGDクラス（Stochastic Gradient Descent、確率的勾配降下法）として作成します。


雛形



In [ ]:
class SGD:
    """
    確率的勾配降下法
    Parameters
    ----------
    lr : 学習率
    """
    def __init__(self, lr):
        self.lr = lr
    def update(self, layer):
        """
        ある層の重みやバイアスの更新
        Parameters
        ----------
        layer : 更新前の層のインスタンス
        """

## 【問題4】活性化関数のクラス化
活性化関数のクラス化を行なってください。


ソフトマックス関数のバックプロパゲーションには交差エントロピー誤差の計算も含む実装を行うことで計算が簡略化されます。


発展的要素
活性化関数や重みの初期値、最適化手法に関してこれまで見てきた以外のものを実装していきます。


【問題5】ReLUクラスの作成
現在一般的に使われている活性化関数であるReLU（Rectified Linear Unit）をReLUクラスとして実装してください。


ReLUは以下の数式です。

$$f(x) = ReLU(x) = \begin{cases}
x  & \text{if $x>0$,}\\
0 & \text{if $x\leqq0$.}
\end{cases}$$

x
 : ある特徴量。スカラー


実装上はnp.maximumを使い配列に対してまとめて計算が可能です。


numpy.maximum — NumPy v1.15 Manual  
https://docs.scipy.org/doc/numpy-1.15.0/reference/generated/numpy.maximum.html



一方、バックプロパゲーションのための 
x
 に関する 
f
(
x
)
 の微分は以下のようになります。

 $$\frac{\partial f(x)}{\partial x} = \begin{cases}
1  & \text{if $x>0$,}\\
0 & \text{if $x\leqq0$.}
\end{cases}$$

数学的には微分可能ではないですが、 
x
=
0
 のとき 
0
 とすることで対応しています。


フォワード時の 
x
 の正負により、勾配を逆伝播するかどうかが決まるということになります。

## 【問題6】重みの初期値
ここまでは重みやバイアスの初期値は単純にガウス分布で、標準偏差をハイパーパラメータとして扱ってきました。しかし、どのような値にすると良いかが知られています。シグモイド関数やハイパボリックタンジェント関数のときは Xavierの初期値 （またはGlorotの初期値）、ReLUのときは Heの初期値 が使われます。


XavierInitializerクラスと、HeInitializerクラスを作成してください。


**Xavierの初期値**  
Xavierの初期値における標準偏差 
σ
 は次の式で求められます。
 $$\sigma = \frac{1}{\sqrt{n}}$$

 n
  : 前の層のノード数


《論文》


Glorot, X., & Bengio, Y. (n.d.). Understanding the difficulty of training deep feedforward neural networks.  
http://proceedings.mlr.press/v9/glorot10a/glorot10a.pdf


Heの初期値
Heの初期値における標準偏差 
σ
 は次の式で求められます。
 $$\sigma = \sqrt{\frac{2}{n}}$$
 
 n
 : 前の層のノード数


《論文》


He, K., Zhang, X., Ren, S., & Sun, J. (2015). Delving Deep into Rectifiers: Surpassing Human-Level Performance on ImageNet Classification.  
 https://arxiv.org/pdf/1502.01852.pdf

## 【問題7】最適化手法
学習率は学習過程で変化させていく方法が一般的です。基本的な手法である AdaGrad のクラスを作成してください。


まず、これまで使ってきたSGDを確認します。

$$W_i^{\prime} = W_i - \alpha E(\frac{\partial L}{\partial W_i}) \\
B_i^{\prime} = B_i - \alpha E(\frac{\partial L}{\partial B_i})$$

$\alpha$ : 学習率（層ごとに変えることも可能だが、基本的には全て同じとする）

$\frac{\partial L}{\partial W_i}$ : $W_i$ に関する損失 $L$ の勾配

$\frac{\partial L}{\partial B_i}$ : $B_i$ に関する損失 $L$ の勾配

$E()$ : ミニバッチ方向にベクトルの平均を計算

続いて、AdaGradです。バイアスの数式は省略しますが、重みと同様のことをします。

更新された分だけその重みに対する学習率を徐々に下げていきます。イテレーションごとの勾配の二乗和 $H$ を保存しておき、その分だけ学習率を小さくします。

学習率は重み一つひとつに対して異なることになります。
$$H_i^{\prime}  = H_i+E(\frac{\partial L}{\partial W_i})×E(\frac{\partial L}{\partial W_i})\\
W_i^{\prime} = W_i - \alpha \frac{1}{\sqrt{H_i^{\prime} }} E(\frac{\partial L}{\partial W_i}) $$

H
i
 : i層目に関して、前のイテレーションまでの勾配の二乗和（初期値は0）


H
′
i
 : 更新した 
H
i

《論文》


Duchi JDUCHI, J., & Singer, Y. (2011). Adaptive Subgradient Methods for Online Learning and Stochastic Optimization * Elad Hazan. Journal of Machine Learning Research (Vol. 12).  
http://www.jmlr.org/papers/volume12/duchi11a/duchi11a.pdf

## 【問題8】クラスの完成
任意の構成で学習と推定が行えるScratchDeepNeuralNetworkClassifierクラスを完成させてください。

# 3.検証

## 【問題9】学習と推定
層の数や活性化関数を変えたいくつかのネットワークを作成してください。そして、MNISTのデータを学習・推定し、Accuracyを計算してください。

# 下記に解答用クラスなどをまとめております。

## 前処理

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as setattr
from sklearn.metrics import accuracy_score

In [ ]:
class GetMiniBatch:
    """
    ミニバッチを取得するイテレータ
    Parameters
    ----------
    X : 次の形のndarray, shape (n_samples, n_features)
      訓練データ
    y : 次の形のndarray, shape (n_samples, 1)
      正解値
    batch_size : int
      バッチサイズ
    seed : int
      NumPyの乱数のシード
    """
    def __init__(self, X, y, batch_size = 20, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self._X = X[shuffle_index]
        self._y = y[shuffle_index]
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)
    def __len__(self):
        return self._stop
    def __getitem__(self,item):
        p0 = item*self.batch_size
        p1 = item*self.batch_size + self.batch_size
        return self._X[p0:p1], self._y[p0:p1]        
    def __iter__(self):
        self._counter = 0
        return self
    def __next__(self):
        if self._counter >= self._stop:
            raise StopIteration()
        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self._X[p0:p1], self._y[p0:p1]

In [ ]:
from keras.datasets import mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# 平滑化（flatten）
X_train_fltten = X_train.reshape(-1, 784)
X_test_fltten = X_test.reshape(-1, 784)

# float化と0or1処理
X_train_flt = X_train_fltten.astype(np.float)
X_test_flt = X_test_fltten.astype(np.float)
X_train_flt /= 255
X_test_flt /= 255

# one hot処理
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
y_train_one_hot = enc.fit_transform(y_train[:, np.newaxis])
y_test_one_hot = enc.transform(y_test[:, np.newaxis])

In [ ]:
# train, testの分割
from sklearn.model_selection import train_test_split
x_train, x_val, y_train_one_hot, y_val_one_hot = train_test_split(np.array(X_train_flt), np.array(y_train_one_hot), test_size=0.2)
print("x_train",x_train.shape, "x_val", x_val.shape, "y_train_one_hot.shape", y_train_one_hot.shape, "y_val_one_hot", y_val_one_hot.shape) # (48000, 784)

### ベースクラス

In [ ]:
class ScratchDeepNeuralNetworkClassifier():
  def __init__(self, epoch_num=1, batch_size=2000, verbose=True): 
        self.epoch = epoch_num
        self.batch_size = batch_size
        self.verbose = verbose

  def fit(self, initializing, act, optimization, X, y, X_val=None, y_val=None, sigma=0.01, lr=0.01, n_features=784, n_nodes1=400, n_nodes2=200, n_output=10):  
        self.initializing = initializing # 初期化クラスの設定
        self.activation= act # 活性化関数クラスの設定
        self.optimization= optimization # 活性化関数クラスの設定
        self.sigma = sigma
        self.lr = lr
        self.n_features = n_features
        self.n_nodes1 = n_nodes1
        self.n_nodes2 = n_nodes2
        self.n_output = n_output
        self.loss = np.zeros(self.epoch)
        self.val_loss = np.zeros(self.epoch)
        self.h = None #最適化クラス adagraidで使用

        # 最適化クラスの設定
        optimizer = self.optimization(self.lr)

        # 全結合層にインスタンスを渡す
        self.FC1 = FC(self.n_features, self.n_nodes1, self.initializing(self.sigma), optimizer)
        self.activation1 = self.activation() #　活性化関数の設定
        self.FC2 = FC(self.n_nodes1, self.n_nodes2, self.initializing(self.sigma), optimizer)
        self.activation2 = self.activation() #　活性化関数の設定
        self.FC3 = FC(self.n_nodes2, self.n_output, self.initializing(self.sigma), optimizer)
        self.activation3 = Softmax()

        # エポック数分の学習
        for i in range(self.epoch):
          # ミニバッチの作成
          get_mini_batch = GetMiniBatch(X, y, self.batch_size)
          # 1エポック（全バッチ）の学習
          for x_min, y_min in get_mini_batch:
            y_hat = self._forward_propagation(x_min)
            print("y_hat2", y_hat[0,:])
            loss = self._back_propagation(X=y_hat, Y=y_min)
            # print("y_hat",y_hat[:1])

          self.loss[i] += loss
          if (type(X_val) != bool):
            self.val = 1
            y_hat_val = self._forward_propagation(X_val)
            loss_val = -np.mean(y_val_one_hot * np.log(y_hat_val + 1e-7))
            self.val_loss[i] += loss_val

          # self.acc_val[i] = accuracy_score(np.argmax(y_val, axis=1), np.argmax(y_hat_val, axis=1))
          # verboseをTrueにした際は学習過程を出力
          if self.verbose :
            print(f"--{i+1}回目~loss~-------\n{self.loss[i]}")
            print(f"--{i+1}回目~loss_val~---\n{self.val_loss[i]}")
            # print(f'epoch:{self.epoch:>3} loss:{self.loss:>8,.3f}')

  # フォワードプロパゲーションの実行
  def _forward_propagation(self, X):
      A1 = self.FC1.forward(X)
      Z1 = self.activation1.forward(A1)
      A2 = self.FC2.forward(Z1)
      Z2 = self.activation2.forward(A2)
      A3 = self.FC3.forward(Z2)
      Z3 = self.activation3.forward(A3)
      return Z3

  # バックプロパゲーションの実行
  def _back_propagation(self, X, Y):
      print("y_hat3", X[0,:])
      dA3, loss = self.activation3.backward(X, Y) # 交差エントロピー誤差とソフトマックスを合わせている
      dZ2 = self.FC3.backward(dA3)
      dA2 = self.activation2.backward(dZ2)
      dZ1 = self.FC2.backward(dA2)
      dA1 = self.activation1.backward(dZ1)
      dZ0 = self.FC1.backward(dA1) # dZ0は使用しない
      return loss

  def predict(self, X):
      y_hat = self._forward_propagation(X)
      return np.argmax(y_hat, axis=1)

  def plot_cost(self):
      plt.title("Num_of_Iteration vs Loss")
      plt.xlabel("Num_of_Iteration")
      plt.ylabel("Loss")
      a = range(self.epoch)
      plt.plot(range(1, self.epoch+1), self.loss, color="b", label="train_loss")
      if self.val ==1:
          plt.plot(range(1, self.epoch+1), self.val_loss, color="orange", label="val_loss")
      plt.grid()
      plt.legend()

### 全結合層のクラス

In [ ]:
class FC:
    def __init__(self, n_nodes1, n_nodes2, initializer, optimizer):
        self.optimizer = optimizer
        self.n_nodes1 = n_nodes1
        self.n_nodes2 = n_nodes2
        self.W = initializer.W(self.n_nodes1, self.n_nodes2)
        self.B = initializer.B(self.n_nodes2)
        # 初期化
        # initializerのメソッドを使い、self.Wとself.Bを初期化する
        # self.update_w = optimizer.update()
        # self.update_b = optimizer.update()

    # フォワードプロパゲーション時の処理
    def forward(self, X):
      self.X = X
      out = X@self.W+self.B
      return out
    
    # バックプロパゲーション時の処理
    def backward(self, dA):
      dZ = dA@(self.W.T)
      dW = (self.X.T)@dA
      dB = np.sum(dA, axis=0)
      
      # 更新
      # self.W = self.update_w(dW, self.W)
      # self.W = self.update_w(dB, self.B)
      self.W = self.optimizer.update(dW, self.W)
      self.B = self.optimizer.update(dB, self.B)
      # print("W", self.W[:100])
      # print("B", self.B[:100])
      return dZ

### 初期化クラス

In [ ]:
# SimpleInitializer ------------------------------------------------------------
class SimpleInitializer:
    def __init__(self, sigma):
        self.sigma = sigma

    def W(self, n_nodes1, n_nodes2):
        W = self.sigma * np.random.randn(n_nodes1, n_nodes2)
        return W

    def B(self, n_nodes2):
        B = self.sigma * np.random.randn(1, n_nodes2)
        return B

# XavierInitializer ------------------------------------------------------------
class XavierInitializer:
    def __init__(self, Xavier):
        self.Xavier = Xavier
        

    def W(self, n_nodes1, n_nodes2):
        W = np.random.randn(n_nodes1, n_nodes2) * np.sqrt(1.0 / n_nodes1)
        return W

    def B(self, n_nodes2):
        B = np.random.randn(1, n_nodes2) * np.sqrt(1.0 / 1.0)
        return B

# He-----------------------------------------------------------------------------
class HeInitializer:
    def __init__(self, He):
        self.He = He

    def W(self, n_nodes1, n_nodes2):
        W = np.random.randn(n_nodes1, n_nodes2) * np.sqrt(2.0 / n_nodes1)
        return W

    def B(self, n_nodes2):
        B = np.random.randn(1, n_nodes2) * np.sqrt(2.0 / 1)
        return B

### 最適化クラス

In [ ]:
# SGD---------------------------------------------
class SGD:
    def __init__(self, lr):
        self.lr = lr

    def update(self, dWorB, WorB):
      self.WorB = WorB
      self.WorB -= self.lr*dWorB
      return self.WorB

# AdaGrad----------------------------------------
class  AdaGrad:
    def __init__(self, lr):
      self.lr = lr
      self.H = 0

    def update(self, dWorB, WorB):
      print("H", self.H)
      self.H += dWorB**2
      print("H", self.H[0,0])
      WorB -= self.lr * dWorB / np.sqrt(self.H)
      return WorB

### 活性化関数クラス

In [ ]:
# ソフトマックス関数のクラス----------------------------------------------
class Softmax:
  def __init__(self):
    pass

  # forward時の処理
  def forward(self, X):
    y_hat = np.exp(X) / np.sum(np.exp(X), axis = 1).reshape(-1,1)
    return y_hat

  # backward時の処理
  def backward(self, X, Y):
    loss = -np.mean(Y * np.log(X + 10e-7))
    print("X", X[0,:])
    print("Y", Y[0,:])
    dA3 = X - Y
    print("dA3", dA3[0,:])
    return dA3, loss

# ReLU関数のクラス----------------------------------------------
class ReLU:
  def __init__(self):
    pass

  # forward時の処理
  def forward(self, X):
    print("re1", X[0,:])
    a = np.maximum(0, X)
    print("re2", a[0,:])
    return a
    
  # backward時の処理
  def backward(self, X):
    print("re3", X[0,:])
    b = np.maximum(0, X)
    print("re2", b[0,:])
    return b
    print("re4", np.maximum(0, X)[0,:])

# tanh関数のクラス----------------------------------------------
class Tanh:
  def __init__(self):
    pass

  # forward時の処理
  def forward(self, X):
    self.out = np.tanh(X)
    return self.out

  # backward時の処理
  def backward(self, X):
    return X*(1-np.tanh(self.out)**2)

### 学習

In [ ]:
sdnn = ScratchDeepNeuralNetworkClassifier(epoch_num=1)
sdnn.fit(XavierInitializer, ReLU, SGD, x_train, y_train_one_hot, x_val, y_val_one_hot, sigma=0.01, lr=0.001)

In [ ]:
y_hat = sdnn.predict(X_test_flt)
y_hat

In [ ]:
y_test[:100]

In [ ]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test, y_hat)

In [ ]:
sdnn.plot_cost()